In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm 

<h6>Raw Public Data Source

In [2]:
csv_path_update = "../data/raw/>150K/public_150k_plus_230630.csv"

In [3]:
ppp_df_23 = pd.read_csv(csv_path_update)
ppp_df_23.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021


2023 - 965552 rows <br>
2021 - 968557 rows <br>
= any previously processed geo, fips DataFrames (matching 2023 Loan Numbers) need to be merged from the left with main 2023 pipe to maintain 965552 row count

In [4]:
ppp_df_23.shape[0]

965552

In [7]:
ppp_df_23.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

<b>Null Check </b><br>
compare null values for address, city, state to original 2021

In [5]:
pd.set_option('display.max_rows', 70)
print(ppp_df_23.isnull().sum())
null_columns=ppp_df_23.columns[ppp_df_23.isnull().any()]
df_null = ppp_df_23[null_columns]
df_null

LoanNumber                          0
DateApproved                        0
SBAOfficeCode                       0
ProcessingMethod                    0
BorrowerName                        4
BorrowerAddress                    14
BorrowerCity                       12
BorrowerState                      13
BorrowerZip                        13
LoanStatusDate                  16825
LoanStatus                          0
Term                                0
SBAGuarantyPercentage               0
InitialApprovalAmount               0
CurrentApprovalAmount               0
UndisbursedAmount                  46
FranchiseName                  930147
ServicingLenderLocationID           0
ServicingLenderName                 0
ServicingLenderAddress              0
ServicingLenderCity                 0
ServicingLenderState                0
ServicingLenderZip                  0
RuralUrbanIndicator                 0
HubzoneIndicator                    0
LMIIndicator                        0
BusinessAgeD

,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,UndisbursedAmount,FranchiseName,BusinessAgeDescription,ProjectCity,...,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,NonProfit,ForgivenessAmount,ForgivenessDate
0,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,0.0,NaN,Existing or more than 2 years old,Sumter,...,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,NaN,773553.37,11/20/2020
1,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,0.0,NaN,Existing or more than 2 years old,North Charleston,...,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,NaN,746336.24,08/12/2021
2,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,0.0,NaN,New Business or 2 years or less,SEATTLE,...,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,Y,696677.49,02/10/2021
3,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,0.0,NaN,New Business or 2 years or less,SAN BERNARDINO,...,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,NaN,395264.11,09/10/2021
4,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,0.0,NaN,Unanswered,NaN,...,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370819.35,04/08/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965547,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601-2724,01/13/2021,0.0,NaN,Existing or more than 2 years old,CASPER,...,150000.00,NaN,NaN,NaN,NaN,NaN,Corporation,NaN,151037.50,12/09/2020
965548,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,82935-5783,12/08/2020,0.0,NaN,Existing or more than 2 years old,GREEN RIVER,...,145000.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Childcare Center,Y,150789.04,11/03/2020
965549,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,82727-8465,09/15/2021,0.0,NaN,Existing or more than 2 years old,Rozet,...,149997.00,NaN,NaN,NaN,NaN,NaN,Subchapter S Corporation,NaN,150743.84,08/17/2021
965550,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,82801,03/08/2022,0.0,NaN,New Business or 2 years or less,SHERIDAN,...,150000.00,NaN,NaN,NaN,NaN,NaN,Corporation,NaN,152820.83,02/23/2022


In [6]:
df_null_sum = ppp_df_23.isnull().sum()
df_null_sum.to_csv("../data/process_data/23rawnull.csv")

In [6]:
pd.reset_option('display.max_rows', silent=True) #disables display option

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/_config/config.py:630: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/_config/config.py:630: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


<h5>Adding Loan Approval Amount Ranges to Determine Lender Percent Profit </h5>
https://home.treasury.gov/system/files/136/PPP%20Lender%20Information%20Fact%20Sheet.pdf

In [7]:
loan_conditions = [
(ppp_df_23.CurrentApprovalAmount <= 350000) & (ppp_df_23.CurrentApprovalAmount >= 150000),
(ppp_df_23.CurrentApprovalAmount <= 2000000) & (ppp_df_23.CurrentApprovalAmount > 350000),
(ppp_df_23.CurrentApprovalAmount > 2000000)
]
loan_values = [.05,.03,.01]
ppp_df_23['Estimated_LenderPercent'] = np.select(loan_conditions, loan_values)
ppp_df_23

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,Estimated_LenderPercent
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020,0.03
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021,0.03
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021,0.03
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021,0.03
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965547,4395967002,04/03/2020,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601-2724,01/13/2021,...,42366,Platte Valley Bank,TORRINGTON,WY,Male Owned,Non-Veteran,NaN,151037.50,12/09/2020,0.05
965548,6985647108,04/14/2020,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,82935-5783,12/08/2020,...,122813,NebraskaLand Bank,NORTH PLATTE,NE,Unanswered,Unanswered,Y,150789.04,11/03/2020,0.05
965549,7996438405,02/12/2021,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,82727-8465,09/15/2021,...,77189,First National Bank of Gillette,GILLETTE,WY,Female Owned,Unanswered,NaN,150743.84,08/17/2021,0.05
965550,9054647103,04/15/2020,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,82801,03/08/2022,...,25901,Small Business Bank,LENEXA,KS,Unanswered,Unanswered,NaN,152820.83,02/23/2022,0.05


In [8]:
ppp_df_23["Estimated_LenderProfit"] = ppp_df_23['CurrentApprovalAmount'] * ppp_df_23['Estimated_LenderPercent']
ppp_df_23["Estimated_LenderProfit"] = ppp_df_23["Estimated_LenderProfit"].apply(lambda x: '{:.2f}'.format(x) if type(x) is int or type(x) is float else x)
ppp_df_23['Estimated_LenderPercent'] = pd.Series(["{0:.0f}%".format(val * 100) for val in ppp_df_23['Estimated_LenderPercent']], index = ppp_df_23.index)
ppp_df_23

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,Estimated_LenderPercent,Estimated_LenderProfit
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020,3%,23080.76
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021,3%,22107.83
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021,3%,20740.65
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021,3%,14996.13
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021,3%,11023.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965547,4395967002,04/03/2020,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601-2724,01/13/2021,...,Platte Valley Bank,TORRINGTON,WY,Male Owned,Non-Veteran,NaN,151037.50,12/09/2020,5%,7500.00
965548,6985647108,04/14/2020,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,82935-5783,12/08/2020,...,NebraskaLand Bank,NORTH PLATTE,NE,Unanswered,Unanswered,Y,150789.04,11/03/2020,5%,7500.00
965549,7996438405,02/12/2021,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,82727-8465,09/15/2021,...,First National Bank of Gillette,GILLETTE,WY,Female Owned,Unanswered,NaN,150743.84,08/17/2021,5%,7500.00
965550,9054647103,04/15/2020,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,82801,03/08/2022,...,Small Business Bank,LENEXA,KS,Unanswered,Unanswered,NaN,152820.83,02/23/2022,5%,7500.00


<b> DTYPES

In [9]:
pd.set_option('display.max_rows', 70)
ppp_df_23.dtypes

LoanNumber                       int64
DateApproved                    object
SBAOfficeCode                    int64
ProcessingMethod                object
BorrowerName                    object
BorrowerAddress                 object
BorrowerCity                    object
BorrowerState                   object
BorrowerZip                     object
LoanStatusDate                  object
LoanStatus                      object
Term                             int64
SBAGuarantyPercentage            int64
InitialApprovalAmount          float64
CurrentApprovalAmount          float64
UndisbursedAmount              float64
FranchiseName                   object
ServicingLenderLocationID        int64
ServicingLenderName             object
ServicingLenderAddress          object
ServicingLenderCity             object
ServicingLenderState            object
ServicingLenderZip              object
RuralUrbanIndicator             object
HubzoneIndicator                object
LMIIndicator             

In [ ]:
pd.reset_option('display.max_rows', silent=True) # disables display option

<b>DATE TO TIMESTAMP CONVERSION 

DateApproved

In [11]:
dates = []
for date in ppp_df_23['DateApproved']:
    dt = pd.to_datetime(date, format='%m/%d/%Y') #year #### format
    dates.append(dt)

ppp_df_23['DateApprovedTS'] = dates
ppp_df_23['DateApprovedTS']= ppp_df_23['DateApprovedTS'].astype(str)
ppp_df_23['DateApprovedTS']= ppp_df_23['DateApprovedTS'].str.replace('-', '', regex=True).astype(int)
ppp_df_23.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020,3%,23080.76,20200501
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021,3%,22107.83,20200501
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021,3%,20740.65,20200501
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021,3%,14996.13,20200501
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021,3%,11023.11,20200501


ForgivenessDate

In [12]:
fdates = []
for fdate in ppp_df_23['ForgivenessDate']:
    fdt = pd.to_datetime(fdate, format='%m/%d/%Y') #year #### format
    fdates.append(fdt)

ppp_df_23['ForgivenessDateTS'] = fdates
ppp_df_23['ForgivenessDateTS']= ppp_df_23['ForgivenessDateTS'].astype(str)
ppp_df_23['ForgivenessDateTS']= ppp_df_23['ForgivenessDateTS'].str.replace('-', '', regex=True)
ppp_df_23.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020,3%,23080.76,20200501,20201120
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021,3%,22107.83,20200501,20210812
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021,3%,20740.65,20200501,20210210
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021,3%,14996.13,20200501,20210910
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021,3%,11023.11,20200501,20210408


In [13]:
ppp_df_23 = ppp_df_23.drop(['DateApproved', 'ForgivenessDate'], axis=1)
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408


In [14]:
pd.set_option('display.max_columns', None)
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510.0,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730.0,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,NaN,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115.0,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210.0,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408


<b>Determining NAICS for Industry of Each Loan

conditions = [
    (ppp_df_23['NA'] == 0),
    (ppp_df_23['NA'] == 99),
    (ppp_df_23['NA'] == 72),
    (ppp_df_23['NA'] == 56),
    (ppp_df_23['NA'] == 11),
    (ppp_df_23['NA'] == 71),
    (ppp_df_23['NA'] == 23),
    (ppp_df_23['NA'] == 61),
    (ppp_df_23['NA'] == 52),
    (ppp_df_23['NA'] == 62),
    (ppp_df_23['NA'] == 51),
    (ppp_df_23['NA'] == 55),
    (ppp_df_23['NA'] == 31),
    (ppp_df_23['NA'] == 32),
    (ppp_df_23['NA'] == 33),
    (ppp_df_23['NA'] == 21),
    (ppp_df_23['NA'] == 81),
    (ppp_df_23['NA'] == 54),
    (ppp_df_23['NA'] == 92),
    (ppp_df_23['NA'] == 53),
    (ppp_df_23['NA'] == 44),
    (ppp_df_23['NA'] == 45),
    (ppp_df_23['NA'] == 48),
    (ppp_df_23['NA'] == 49),
    (ppp_df_23['NA'] == 22),
    (ppp_df_23['NA'] == 42)
]

values = ['0',
 '99 - Nonclassifiable',
 'Accomodation and Food Services',
 'Administrative and Support and Waste Management and Remediation Services',
 'Agriculture, Forestry, Fishing and Hunting',
 'Arts, Entertainment, and Recreation',
 'Construction',
 'Educational Services',
 'Finance and Insurance',
 'Health Care and Social Assistance',
 'Information',
 'Management of Companies and Enterprises',
 'Manufacturing',
 'Manufacturing',
 'Manufacturing',
 'Mining',
 'Other Services (except Public Administration)',
 'Professional, Scientific, and Technical Services',
 'Public Administration',
 'Real Estate Rental and Leasing',
 'Retail Trade',
 'Retail Trade',
 'Transportation and Warehousing',
 'Transportation and Warehousing',
 'Utilities',
 'Wholesale Trade']

ppp_df_23['Industry'] = np.select(conditions, values)

ppp_df_23.head()

In [15]:
naics_ind = pd.read_csv("../data/raw/NAICS/NAICSind.csv") #NAICSCode, Industry
naics_full = pd.read_csv("../data/raw/NAICS/NAICSfull.csv") #NAICSCode, SubIndustry Values

In [16]:
naics_ind["NA"] = naics_ind["NA"].astype(str)
naics_ind.head()

,NA,Industry
0,11,"Agriculture, Forestry, Fishing and Hunting"
1,21,"Mining, Quarrying, and Oil and Gas Extraction"
2,22,Utilities
3,23,Construction
4,31,Manufacturing


In [17]:
naics_full["NA"] = naics_full["NAICSCode"].astype(str).str[:2]
naics_full 

,NAICSCode,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA
0,111110,Soybean Farming,999 Other,1111 Oilseed and Grain Farming,1111,11
1,111120,Oilseed (except Soybean) Farming,999 Other,1111 Oilseed and Grain Farming,1111,11
2,111130,Dry Pea and Bean Farming,999 Other,1111 Oilseed and Grain Farming,1111,11
3,111140,Wheat Farming,999 Other,1111 Oilseed and Grain Farming,1111,11
4,111150,Corn Farming,999 Other,1111 Oilseed and Grain Farming,1111,11
...,...,...,...,...,...,...
1052,926140,Regulation of Agricultural Marketing and Commo...,999 Other,9261 Administration of Economic Programs,9261,92
1053,926150,"Regulation, Licensing, and Inspection of Misce...",999 Other,9261 Administration of Economic Programs,9261,92
1054,927110,Space Research and Technology,999 Other,9271 Space Research and Technology,9271,92
1055,928110,National Security,999 Other,9281 National Security and International Affairs,9281,92


In [18]:
naics_full = naics_full.merge(naics_ind, on ="NA", how="left")
naics_full

,NAICSCode,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
0,111110,Soybean Farming,999 Other,1111 Oilseed and Grain Farming,1111,11,"Agriculture, Forestry, Fishing and Hunting"
1,111120,Oilseed (except Soybean) Farming,999 Other,1111 Oilseed and Grain Farming,1111,11,"Agriculture, Forestry, Fishing and Hunting"
2,111130,Dry Pea and Bean Farming,999 Other,1111 Oilseed and Grain Farming,1111,11,"Agriculture, Forestry, Fishing and Hunting"
3,111140,Wheat Farming,999 Other,1111 Oilseed and Grain Farming,1111,11,"Agriculture, Forestry, Fishing and Hunting"
4,111150,Corn Farming,999 Other,1111 Oilseed and Grain Farming,1111,11,"Agriculture, Forestry, Fishing and Hunting"
...,...,...,...,...,...,...,...
1052,926140,Regulation of Agricultural Marketing and Commo...,999 Other,9261 Administration of Economic Programs,9261,92,Public Administration
1053,926150,"Regulation, Licensing, and Inspection of Misce...",999 Other,9261 Administration of Economic Programs,9261,92,Public Administration
1054,927110,Space Research and Technology,999 Other,9271 Space Research and Technology,9271,92,Public Administration
1055,928110,National Security,999 Other,9281 National Security and International Affairs,9281,92,Public Administration


In [19]:
print(naics_full.isnull().sum())

NAICSCode            0
LongName             0
TRIIndustrySector    0
IndustrySubsector    0
4DigitNAICS          0
NA                   0
Industry             0
dtype: int64


In [20]:
naics_full.dtypes

NAICSCode             int64
LongName             object
TRIIndustrySector    object
IndustrySubsector    object
4DigitNAICS           int64
NA                   object
Industry             object
dtype: object

In [21]:
ppp_df_23.dtypes

LoanNumber                       int64
SBAOfficeCode                    int64
ProcessingMethod                object
BorrowerName                    object
BorrowerAddress                 object
BorrowerCity                    object
BorrowerState                   object
BorrowerZip                     object
LoanStatusDate                  object
LoanStatus                      object
Term                             int64
SBAGuarantyPercentage            int64
InitialApprovalAmount          float64
CurrentApprovalAmount          float64
UndisbursedAmount              float64
FranchiseName                   object
ServicingLenderLocationID        int64
ServicingLenderName             object
ServicingLenderAddress          object
ServicingLenderCity             object
ServicingLenderState            object
ServicingLenderZip              object
RuralUrbanIndicator             object
HubzoneIndicator                object
LMIIndicator                    object
BusinessAgeDescription   

Convert Column Type Before Merge

In [22]:
ppp_df_23["NAICSCode"] = ppp_df_23["NAICSCode"].fillna(0)
ppp_df_23["NAICSCode"] = ppp_df_23["NAICSCode"].astype(int)
ppp_df_23

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965547,4395967002,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601-2724,01/13/2021,Paid in Full,24,100,150000.00,150000.00,0.0,NaN,42366,Platte Valley Bank,2201 Main St,TORRINGTON,WY,82240-2317,U,N,N,Existing or more than 2 years old,CASPER,NATRONA,WY,82601-2724,WY-,15.0,621210,Unanswered,Not Hispanic or Latino,NaN,150000.00,NaN,NaN,NaN,NaN,NaN,Corporation,42366,Platte Valley Bank,TORRINGT

In [23]:
ppp_df_23_merge = ppp_df_23.merge(naics_full, on="NAICSCode", how="left")
ppp_df_23_merge #6632

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120,Paint and Coating Manufacturing,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,32,Manufacturing
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812,Landscaping Services,999 Other,5617 Services to Buildings and Dwellings,5617.0,56,Administrative and Support and Waste Managemen...
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,NaN,NaN,NaN,NaN,NaN,NaN
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910,New Single-Family Housing Construction (except...,999 Other,2361 Residential Building Construction,2361.0,23,Construction
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408,Used Household and Office Goods Moving,999 Other,4842 Specialized Freight Trucking,4842.0,48,Transportation and Warehousing
...,...,...,...,...,...,...,...,...,...,..

In [24]:
print(ppp_df_23_merge.isnull().sum())

LoanNumber               0
SBAOfficeCode            0
ProcessingMethod         0
BorrowerName             4
BorrowerAddress         14
                     ...  
TRIIndustrySector    12162
IndustrySubsector    12162
4DigitNAICS          12162
NA                   12162
Industry             12162
Length: 61, dtype: int64


Review Nulls on Merge [Raw Data NAICS shows # missing NAICSCodes]

In [25]:
ppp_df_23_merge.loc[ppp_df_23_merge["Industry"].isnull()]

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,NaN,NaN,NaN,NaN,NaN,NaN
36,4820397010,1084,PPP,"COLVILLE, INC.",4300 B Street,ANCHORAGE,AK,99503-5913,07/20/2021,Paid in Full,24,100,3776300.00,3776300.00,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,Y,Y,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99503-5913,AK-,213.0,454319,White,Not Hispanic or Latino,NaN,3776300.0,NaN,NaN,NaN,NaN,NaN,Corporation,116975,Northrim Bank,ANCHORAGE,AK,Male Owned,Non-Veteran,NaN,3819960.24,1%,37763.00,20200404,20210611,NaN,NaN,NaN,NaN,NaN,NaN
260,6287537004,1084,PPP,SEWARD ASSOCIATION FOR THE ADVANCEMENT OF MARI...,301 Railway Avenue,SEWARD,AK,99664-3801,12/17/2020,Paid in Full,24,100,1046000.00,1046000.00,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,Y,N,Existing or more than 2 years old,SEWARD,KENAI PENINSULA,AK,99664-3801,AK-,75.0,541712,Unanswered,Unknown/NotStated,NaN,1046000.0,NaN,NaN,NaN,NaN,NaN,Professional Association,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,NaN,1051530.90,3%,31380.00,20200406,20201103,NaN,NaN,NaN,NaN,NaN,NaN
305,1892977209,1084,PPP,DENALI VISIONS 3000,2211 Sunrise Dr,ANCHORAGE,AK,99508-3273,01/06/2022,Paid in Full,24,100,425049.37,959270.37,0.0,NaN,3386,First National Bank Alaska,101 W 36th Ave,ANCHORAGE,AK,99503-5904,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99508-3273,AK-,107.0,722110,White,Not Hispanic or Latino,14926.0,874260.5,NaN,70083.87,NaN,NaN,NaN,Corporation,3386,First National Bank Alaska,ANCHORAGE,AK,Male Owned,Non-Veteran,NaN,974986.64,3%,28778.11,20200415,20211214,NaN,NaN,NaN,NaN,NaN,NaN
429,5784067001,1084,PPP,TONGASS TRADING COMPANY,201 DOCK ST,KETCHIKAN,AK,99901-6407,01/29/2021,Paid in Full,24,100,738072.00,738072.00,0.0,NaN,3501,First Bank,2030 Sea level Dr Ste 300,KETCHIKAN,AK,99901-6440,U,Y,N,Existing or more than 2 years old,KETCHIKAN,KETCHIKAN GATEWAY,AK,99901-6407,AK-,48.0,452990,Unanswered,Unknown/NotStated,NaN,738072.0,NaN,NaN,NaN,NaN,NaN,Corporation,3501,First Bank,KETCHIKAN,AK,Male Owned,Unanswered,NaN,742358.88,3%,22142.16,20200406,20201105,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964869,4935817005,897,PPP,"BEG, LLC",1709 Muirfield Court PO BOX 319,DOUGLAS,WY,82633-8701,02/18/2021,Paid in Full,24,100,213400.00,213400.00,0.0,NaN,77183,The Converse County Bank,322 Walnut

<b>NAICS Formatting

In [26]:
ppp_df_23 = ppp_df_23_merge

In [27]:
#ppp_df_23["NAICSCode"] = ppp_df_23["NAICSCode"].fillna(0).astype(np.int64).astype(str)
ppp_df_23["NA"] = ppp_df_23["NA"].fillna(0).astype(np.int64).astype(str)
ppp_df_23["4DigitNAICS"] = ppp_df_23["4DigitNAICS"].fillna(0).astype(np.int64).astype(str).str[:4]
ppp_df_23["Industry"] = ppp_df_23["Industry"].fillna("Not Available")
ppp_df_23["IndustrySubsector"] = ppp_df_23["IndustrySubsector"].fillna("Not Available")
ppp_df_23["LongName"] = ppp_df_23["IndustrySubsector"].fillna("Not Available")
ppp_df_23

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,Not Available,NaN,Not Available,0,0,Not Available
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing
..

999990 NonClassifiable Establishments - Official Label 

In [28]:
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990]

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
1503,1126217303,1084,PPP,"NYSTUENS FOLLY, LLC",1811 Abbott Rd,ANCHORAGE,AK,99507,08/07/2021,Paid in Full,24,100,251520.83,251520.83,0.0,NaN,3386,First National Bank Alaska,101 W 36th Ave,ANCHORAGE,AK,99503-5904,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99507-0001,AK-,31.0,999990,White,Unknown/NotStated,23697.0,191718.83,NaN,36105.0,NaN,NaN,NaN,Limited Liability Company(LLC),3386,First National Bank Alaska,ANCHORAGE,AK,Female Owned,Non-Veteran,NaN,254601.10,5%,12576.04,20200428,20210722,Not Available,NaN,Not Available,0,0,Not Available
2001,5414867008,1084,PPP,ALASKA SB LLC,317 W 104th Ave.,ANCHORAGE,AK,99515-2611,04/20/2021,Paid in Full,24,100,197000.00,197000.00,0.0,Smashburger,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99515-2611,AK-,39.0,999990,Unanswered,Unknown/NotStated,NaN,197000.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,NaN,198845.86,5%,9850.00,20200405,20210331,Not Available,NaN,Not Available,0,0,Not Available
2218,4678427701,1084,PPP,HALCYON DAYS LLC,41630 GLADYS COURT,HOMER,AK,99603,12/16/2021,Paid in Full,24,100,17006.00,178526.00,0.0,NaN,12096,"Wells Fargo Bank, National Association",101 N Philips Ave,SIOUX FALLS,SD,57104-6738,U,Y,N,New Business or 2 years or less,HOMER,KENAI PENINSULA,AK,99603-0001,AK-,3.0,999990,Unanswered,Unknown/NotStated,NaN,178526.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),12096,"Wells Fargo Bank, National Association",SIOUX FALLS,SD,Male Owned,Non-Veteran,NaN,162872.34,5%,8926.30,20200501,20211006,Not Available,NaN,Not Available,0,0,Not Available
3526,7212977105,459,PPP,"NEAL TECHNICAL INNOVATIONS, LLC",9101 Milton Jones Road,DAPHNE,AL,36526-6226,04/06/2021,Paid in Full,24,100,1199700.00,1199700.00,0.0,NaN,39232,Hancock Whitney Bank,2510 14th St One Hancock Plz,GULFPORT,MS,39501,R,N,N,Existing or more than 2 years old,DAPHNE,BALDWIN,AL,36526-6226,AL-01,60.0,999990,Unanswered,Unknown/NotStated,NaN,1199700.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),39232,Hancock Whitney Bank,GULFPORT,MS,Unanswered,Unanswered,NaN,1210597.27,3%,35991.00,20200414,20210317,Not Available,NaN,Not Available,0,0,Not Available
4702,5062267000,459,PPP,PAY SUITE LLC,611 Walnut Street,GADSDEN,AL,35901-4136,01/07/2021,Paid in Full,24,100,658500.00,658500.00,0.0,NaN,2037,Peoples Bank of Alabama,1912 Cherokee Ave SW,CULLMAN,AL,35055-5595,R,Y,N,Existing or more than 2 years old,GADSDEN,ETOWAH,AL,35901-4136,AL-04,57.0,999990,White,Not Hispanic or Latino,NaN,658500.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),2037,Peoples Bank of Alabama,CULLMAN,AL,Female Owned,Non-Veteran,NaN,663329.00,3%,19755.00,20200404,20201230,Not Available,NaN,Not Available,0,0,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [29]:
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990, 'Industry'] = 'Nonclassifiable Establishments'
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990, 'IndustrySubsector'] = 'Nonclassifiable Establishments'
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990, 'NA'] = 99
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990, '4DigitNAICS'] = 9999
ppp_df_23

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,Not Available,NaN,Not Available,0,0,Not Available
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing
..

In [30]:
ppp_df_23.loc[ppp_df_23["NAICSCode"] == 999990].head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
1503,1126217303,1084,PPP,"NYSTUENS FOLLY, LLC",1811 Abbott Rd,ANCHORAGE,AK,99507,08/07/2021,Paid in Full,24,100,251520.83,251520.83,0.0,NaN,3386,First National Bank Alaska,101 W 36th Ave,ANCHORAGE,AK,99503-5904,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99507-0001,AK-,31.0,999990,White,Unknown/NotStated,23697.0,191718.83,NaN,36105.0,NaN,NaN,NaN,Limited Liability Company(LLC),3386,First National Bank Alaska,ANCHORAGE,AK,Female Owned,Non-Veteran,NaN,254601.10,5%,12576.04,20200428,20210722,Not Available,NaN,Nonclassifiable Establishments,9999,99,Nonclassifiable Establishments
2001,5414867008,1084,PPP,ALASKA SB LLC,317 W 104th Ave.,ANCHORAGE,AK,99515-2611,04/20/2021,Paid in Full,24,100,197000.00,197000.00,0.0,Smashburger,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99515-2611,AK-,39.0,999990,Unanswered,Unknown/NotStated,NaN,197000.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,NaN,198845.86,5%,9850.00,20200405,20210331,Not Available,NaN,Nonclassifiable Establishments,9999,99,Nonclassifiable Establishments
2218,4678427701,1084,PPP,HALCYON DAYS LLC,41630 GLADYS COURT,HOMER,AK,99603,12/16/2021,Paid in Full,24,100,17006.00,178526.00,0.0,NaN,12096,"Wells Fargo Bank, National Association",101 N Philips Ave,SIOUX FALLS,SD,57104-6738,U,Y,N,New Business or 2 years or less,HOMER,KENAI PENINSULA,AK,99603-0001,AK-,3.0,999990,Unanswered,Unknown/NotStated,NaN,178526.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),12096,"Wells Fargo Bank, National Association",SIOUX FALLS,SD,Male Owned,Non-Veteran,NaN,162872.34,5%,8926.30,20200501,20211006,Not Available,NaN,Nonclassifiable Establishments,9999,99,Nonclassifiable Establishments
3526,7212977105,459,PPP,"NEAL TECHNICAL INNOVATIONS, LLC",9101 Milton Jones Road,DAPHNE,AL,36526-6226,04/06/2021,Paid in Full,24,100,1199700.00,1199700.00,0.0,NaN,39232,Hancock Whitney Bank,2510 14th St One Hancock Plz,GULFPORT,MS,39501,R,N,N,Existing or more than 2 years old,DAPHNE,BALDWIN,AL,36526-6226,AL-01,60.0,999990,Unanswered,Unknown/NotStated,NaN,1199700.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),39232,Hancock Whitney Bank,GULFPORT,MS,Unanswered,Unanswered,NaN,1210597.27,3%,35991.00,20200414,20210317,Not Available,NaN,Nonclassifiable Establishments,9999,99,Nonclassifiable Establishments
4702,5062267000,459,PPP,PAY SUITE LLC,611 Walnut Street,GADSDEN,AL,35901-4136,01/07/2021,Paid in Full,24,100,658500.00,658500.00,0.0,NaN,2037,Peoples Bank of Alabama,1912 Cherokee Ave SW,CULLMAN,AL,35055-5595,R,Y,N,Existing or more than 2 years old,GADSDEN,ETOWAH,AL,35901-4136,AL-04,57.0,999990,White,Not Hispanic or Latino,NaN,658500.00,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),2037,Peoples Bank of Alabama,CULLMAN,AL,Female Owned,Non-Veteran,NaN,663329.00,3%,19755.00,20200404,20201230,Not Available,NaN,Nonclassifiable Establishments,9999,99,Nonclassifiable Establishments


In [31]:
ppp_df_23.loc[ppp_df_23["4DigitNAICS"]=='0'].head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,Not Available,NaN,Not Available,0,0,Not Available
36,4820397010,1084,PPP,"COLVILLE, INC.",4300 B Street,ANCHORAGE,AK,99503-5913,07/20/2021,Paid in Full,24,100,3776300.00,3776300.00,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,Y,Y,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99503-5913,AK-,213.0,454319,White,Not Hispanic or Latino,NaN,3776300.0,NaN,NaN,NaN,NaN,NaN,Corporation,116975,Northrim Bank,ANCHORAGE,AK,Male Owned,Non-Veteran,NaN,3819960.24,1%,37763.00,20200404,20210611,Not Available,NaN,Not Available,0,0,Not Available
260,6287537004,1084,PPP,SEWARD ASSOCIATION FOR THE ADVANCEMENT OF MARI...,301 Railway Avenue,SEWARD,AK,99664-3801,12/17/2020,Paid in Full,24,100,1046000.00,1046000.00,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,Y,N,Existing or more than 2 years old,SEWARD,KENAI PENINSULA,AK,99664-3801,AK-,75.0,541712,Unanswered,Unknown/NotStated,NaN,1046000.0,NaN,NaN,NaN,NaN,NaN,Professional Association,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,NaN,1051530.90,3%,31380.00,20200406,20201103,Not Available,NaN,Not Available,0,0,Not Available
305,1892977209,1084,PPP,DENALI VISIONS 3000,2211 Sunrise Dr,ANCHORAGE,AK,99508-3273,01/06/2022,Paid in Full,24,100,425049.37,959270.37,0.0,NaN,3386,First National Bank Alaska,101 W 36th Ave,ANCHORAGE,AK,99503-5904,U,N,N,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99508-3273,AK-,107.0,722110,White,Not Hispanic or Latino,14926.0,874260.5,NaN,70083.87,NaN,NaN,NaN,Corporation,3386,First National Bank Alaska,ANCHORAGE,AK,Male Owned,Non-Veteran,NaN,974986.64,3%,28778.11,20200415,20211214,Not Available,NaN,Not Available,0,0,Not Available
429,5784067001,1084,PPP,TONGASS TRADING COMPANY,201 DOCK ST,KETCHIKAN,AK,99901-6407,01/29/2021,Paid in Full,24,100,738072.00,738072.00,0.0,NaN,3501,First Bank,2030 Sea level Dr Ste 300,KETCHIKAN,AK,99901-6440,U,Y,N,Existing or more than 2 years old,KETCHIKAN,KETCHIKAN GATEWAY,AK,99901-6407,AK-,48.0,452990,Unanswered,Unknown/NotStated,NaN,738072.0,NaN,NaN,NaN,NaN,NaN,Corporation,3501,First Bank,KETCHIKAN,AK,Male Owned,Unanswered,NaN,742358.88,3%,22142.16,20200406,20201105,Not Available,NaN,Not Available,0,0,Not Available


In [32]:
ppp_df_23["4DigitNAICS"] = ppp_df_23.apply(lambda x: x["NA"] + '000' if x["4DigitNAICS"]=='0' else x["4DigitNAICS"], axis=1)
ppp_df_23["NA"] = ppp_df_23.apply(lambda x: x["NA"] + '0' if x["NA"]=='0' else x["NA"], axis=1)
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,3%,23080.76,20200501,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,3%,22107.83,20200501,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,0,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,3%,20740.65,20200501,20210210,Not Available,NaN,Not Available,0000,00,Not Available
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,24,100,499871.00,499871.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SAN BERNARDINO,SAN BERNARDINO,CA,92407-1740,CA-23,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,NaN,499871.00,NaN,NaN,NaN,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,3%,14996.13,20200501,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,24,100,367437.00,367437.00,0.0,NaN,57328,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,U,N,N,Unanswered,NaN,NaN,NaN,NaN,NaN,25.0,484210,Unanswered,Unknown/NotStated,NaN,367437.00,NaN,NaN,NaN,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,3%,11023.11,20200501,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousin

In [33]:
ppp_df_23.loc[ppp_df_23["4DigitNAICS"].isnull()]

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,Estimated_LenderPercent,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry


In [34]:
pd.set_option('display.max_rows', 70)
print(ppp_df_23.isnull().sum())

LoanNumber                          0
SBAOfficeCode                       0
ProcessingMethod                    0
BorrowerName                        4
BorrowerAddress                    14
BorrowerCity                       12
BorrowerState                      13
BorrowerZip                        13
LoanStatusDate                  16825
LoanStatus                          0
Term                                0
SBAGuarantyPercentage               0
InitialApprovalAmount               0
CurrentApprovalAmount               0
UndisbursedAmount                  46
FranchiseName                  930147
ServicingLenderLocationID           0
ServicingLenderName                 0
ServicingLenderAddress              0
ServicingLenderCity                 0
ServicingLenderState                0
ServicingLenderZip                  0
RuralUrbanIndicator                 0
HubzoneIndicator                    0
LMIIndicator                        0
BusinessAgeDescription              1
ProjectCity 

In [35]:
pd.reset_option('display.max_rows', silent=True) # disables display option

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/_config/config.py:630: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/_config/config.py:630: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


Determining SBA District Office for Processing Each Loan

SBA OFFICE CODE LABELS
0100	  	BOSTON REGIONAL OFFICE
0101	  	MASSACHUSETTS DISTRICT OFFICE
0130	  	SPRINGFIELD BRANCH OFFICE
0150	  	VERMONT DISTRICT OFFICE
0156	  	CONNECTICUT DISTRICT OFFICE
0165	  	RHODE ISLAND DISTRICT OFFICE
0172	  	MAINE DISTRICT OFFICE
0189	  	NEW HAMPSHIRE DISTRICT OFFICE
0200	  	NEW YORK REGIONAL OFFICE
0202	  	NEW YORK DISTRICT OFFICE
0206	  	ELMIRA BRANCH OFFICE
0219	  	ROCHESTER BRANCH OFFICE
0235	  	MELVILLE BRANCH OFFICE
0248	  	SYRACUSE DISTRICT OFFICE
0252	  	PUERTO RICO AND USVI DISTRICT OFFICE
0296	  	BUFFALO DISTRICT OFFICE
0299	  	NEW JERSEY DISTRICT OFFICE
0300	  	PHILADELPHIA REGIONAL OFFICE
0303	  	PHILADELPHIA DISTRICT OFFICE
0304	  	RICHMOND DISTRICT OFFICE
0316	  	HARRISBURG BRANCH OFFICE
0318	  	WILKES BARRE BRANCH OFFICE
0325	  	CHARLESTON BRANCH OFFICE
0341	  	DELAWARE DISTRICT OFFICE
0353	  	WASHINGTON DISTRICT OFFICE
0358	  	PITTSBURGH DISTRICT OFFICE
0373	  	BALTIMORE DISTRICT OFFICE
0390	  	WEST VIRGINIA DISTRICT OFFICE
0400	  	ATLANTA REGIONAL OFFICE
0405	  	GEORGIA DISTRICT OFFICE
0438	  	GULFPORT
0455	  	SOUTH FLORIDA DISTRICT OFFICE
0457	  	KENTUCKY DISTRICT OFFICE
0459	  	ALABAMA DISTRICT OFFICE
0460	  	NORTH CAROLINA DISTRICT OFFICE
0464	  	SOUTH CAROLINA DISTRICT OFFICE
0470	  	MISSISSIPPI DISTRICT OFFICE
0474	  	TENNESSEE DISTRICT OFFICE
0491	  	NORTH FLORIDA DISTRICT OFFICE
0500	  	CHICAGO REGIONAL OFFICE
0507	  	ILLINOIS DISTRICT OFFICE
0508	  	MINNESOTA DISTRICT OFFICE
0515	  	MICHIGAN DISTRICT OFFICE
0517	  	SPRINGFIELD BRANCH OFFICE
0545	  	CINCINNATI BRANCH OFFICE
0547	  	MARQUETTE BRANCH OFFICE
0549	  	CLEVELAND DISTRICT OFFICE
0562	  	INDIANA DISTRICT OFFICE
0563	  	WISCONSIN DISTRICT OFFICE
0593	  	COLUMBUS DISTRICT OFFICE
0600	  	DALLAS REGIONAL OFFICE
0610	  	DALLAS / FT WORTH DISTRICT OFFICE
0637	  	CORPUS CHRISTI BRANCH OFFICE
0639	  	LOWER RIO GRANDE VALLEY DISTRICT OFFICE
0669	  	ARKANSAS DISTRICT OFFICE
0671	  	HOUSTON DISTRICT OFFICE
0677	  	EL PASO DISTRICT OFFICE
0678	  	LUBBOCK DISTRICT OFFICE
0679	  	LOUISIANA DISTRICT OFFICE
0680	  	OKLAHOMA DISTRICT OFFICE
0681	  	SAN ANTONIO DISTRICT OFFICE
0682	  	NEW MEXICO DISTRICT OFFICE
0700	  	KANSAS CITY REGIONAL OFFICE
0709	  	KANSAS CITY DISTRICT OFFICE
0721	  	SPRINGFIELD BRANCH OFFICE
0736	  	CEDAR RAPIDS BRANCH OFFICE
0761	  	DES MOINES DISTRICT OFFICE
0766	  	NEBRASKA DISTRICT OFFICE
0767	  	WICHITA DISTRICT OFFICE
0768	  	ST. LOUIS DISTRICT OFFICE
0800	  	DENVER REGIONAL OFFICE
0811	  	COLORADO DISTRICT OFFICE
0875	  	NORTH DAKOTA DISTRICT OFFICE
0876	  	SOUTH DAKOTA DISTRICT OFFICE
0883	  	UTAH DISTRICT OFFICE
0885	  	MONTANA DISTRICT OFFICE
0897	  	WYOMING DISTRICT OFFICE
0900	  	SAN FRANCISCO REGIONAL OFFICE
0912	  	SAN FRANCISCO DISTRICT OFFICE
0914	  	LOS ANGELES DISTRICT OFFICE
0920	  	SANTA ANA DISTRICT OFFICE
0931	  	SACRAMENTO DISTRICT OFFICE
0942	  	FRESNO DISTRICT OFFICE
0944	  	NEVADA DISTRICT OFFICE
0951	  	HAWAII DISTRICT OFFICE
0954	  	SAN DIEGO DISTRICT OFFICE
0988	  	ARIZONA DISTRICT OFFICE
0995	  	GUAM BRANCH OFFICE
1000	  	SEATTLE REGIONAL OFFICE
1013	  	SEATTLE DISTRICT OFFICE
1084	  	ALASKA DISTRICT OFFICE
1086	  	PORTLAND DISTRICT OFFICE
1087	  	BOISE DISTRICT OFFICE
1094	  	SPOKANE BRANCH OFFICE

In [36]:
conditions = [
(ppp_df_23['SBAOfficeCode'] ==100),
(ppp_df_23['SBAOfficeCode'] ==101),
(ppp_df_23['SBAOfficeCode'] ==130),
(ppp_df_23['SBAOfficeCode'] ==150),
(ppp_df_23['SBAOfficeCode'] ==156),
(ppp_df_23['SBAOfficeCode'] ==165),
(ppp_df_23['SBAOfficeCode'] ==172),
(ppp_df_23['SBAOfficeCode'] ==189),
(ppp_df_23['SBAOfficeCode'] ==200),
(ppp_df_23['SBAOfficeCode'] ==202),
(ppp_df_23['SBAOfficeCode'] ==206),
(ppp_df_23['SBAOfficeCode'] ==219),
(ppp_df_23['SBAOfficeCode'] ==235),
(ppp_df_23['SBAOfficeCode'] ==248),
(ppp_df_23['SBAOfficeCode'] ==252),
(ppp_df_23['SBAOfficeCode'] ==296),
(ppp_df_23['SBAOfficeCode'] ==299),
(ppp_df_23['SBAOfficeCode'] ==300),
(ppp_df_23['SBAOfficeCode'] ==303),
(ppp_df_23['SBAOfficeCode'] ==304),
(ppp_df_23['SBAOfficeCode'] ==316),
(ppp_df_23['SBAOfficeCode'] ==318),
(ppp_df_23['SBAOfficeCode'] ==325),
(ppp_df_23['SBAOfficeCode'] ==341),
(ppp_df_23['SBAOfficeCode'] ==353),
(ppp_df_23['SBAOfficeCode'] ==358),
(ppp_df_23['SBAOfficeCode'] ==373),
(ppp_df_23['SBAOfficeCode'] ==390),
(ppp_df_23['SBAOfficeCode'] ==400),
(ppp_df_23['SBAOfficeCode'] ==405),
(ppp_df_23['SBAOfficeCode'] ==438),
(ppp_df_23['SBAOfficeCode'] ==455),
(ppp_df_23['SBAOfficeCode'] ==457),
(ppp_df_23['SBAOfficeCode'] ==459),
(ppp_df_23['SBAOfficeCode'] ==460),
(ppp_df_23['SBAOfficeCode'] ==464),
(ppp_df_23['SBAOfficeCode'] ==470),
(ppp_df_23['SBAOfficeCode'] ==474),
(ppp_df_23['SBAOfficeCode'] ==491),
(ppp_df_23['SBAOfficeCode'] ==500),
(ppp_df_23['SBAOfficeCode'] ==507),
(ppp_df_23['SBAOfficeCode'] ==508),
(ppp_df_23['SBAOfficeCode'] ==515),
(ppp_df_23['SBAOfficeCode'] ==517),
(ppp_df_23['SBAOfficeCode'] ==545),
(ppp_df_23['SBAOfficeCode'] ==547),
(ppp_df_23['SBAOfficeCode'] ==549),
(ppp_df_23['SBAOfficeCode'] ==562),
(ppp_df_23['SBAOfficeCode'] ==563),
(ppp_df_23['SBAOfficeCode'] ==593),
(ppp_df_23['SBAOfficeCode'] ==600),
(ppp_df_23['SBAOfficeCode'] ==610),
(ppp_df_23['SBAOfficeCode'] ==637),
(ppp_df_23['SBAOfficeCode'] ==639),
(ppp_df_23['SBAOfficeCode'] ==669),
(ppp_df_23['SBAOfficeCode'] ==671),
(ppp_df_23['SBAOfficeCode'] ==677),
(ppp_df_23['SBAOfficeCode'] ==678),
(ppp_df_23['SBAOfficeCode'] ==679),
(ppp_df_23['SBAOfficeCode'] ==680),
(ppp_df_23['SBAOfficeCode'] ==681),
(ppp_df_23['SBAOfficeCode'] ==682),
(ppp_df_23['SBAOfficeCode'] ==700),
(ppp_df_23['SBAOfficeCode'] ==709),
(ppp_df_23['SBAOfficeCode'] ==721),
(ppp_df_23['SBAOfficeCode'] ==736),
(ppp_df_23['SBAOfficeCode'] ==761),
(ppp_df_23['SBAOfficeCode'] ==766),
(ppp_df_23['SBAOfficeCode'] ==767),
(ppp_df_23['SBAOfficeCode'] ==768),
(ppp_df_23['SBAOfficeCode'] ==800),
(ppp_df_23['SBAOfficeCode'] ==811),
(ppp_df_23['SBAOfficeCode'] ==875),
(ppp_df_23['SBAOfficeCode'] ==876),
(ppp_df_23['SBAOfficeCode'] ==883),
(ppp_df_23['SBAOfficeCode'] ==885),
(ppp_df_23['SBAOfficeCode'] ==897),
(ppp_df_23['SBAOfficeCode'] ==900),
(ppp_df_23['SBAOfficeCode'] ==912),
(ppp_df_23['SBAOfficeCode'] ==914),
(ppp_df_23['SBAOfficeCode'] ==920),
(ppp_df_23['SBAOfficeCode'] ==931),
(ppp_df_23['SBAOfficeCode'] ==942),
(ppp_df_23['SBAOfficeCode'] ==944),
(ppp_df_23['SBAOfficeCode'] ==951),
(ppp_df_23['SBAOfficeCode'] ==954),
(ppp_df_23['SBAOfficeCode'] ==988),
(ppp_df_23['SBAOfficeCode'] ==995),
(ppp_df_23['SBAOfficeCode'] ==1000),
(ppp_df_23['SBAOfficeCode'] ==1013),
(ppp_df_23['SBAOfficeCode'] ==1084),
(ppp_df_23['SBAOfficeCode'] ==1086),
(ppp_df_23['SBAOfficeCode'] ==1087),
(ppp_df_23['SBAOfficeCode'] ==1094),
]

values = [
"BOSTON REGIONAL OFFICE",
"MASSACHUSETTS DISTRICT OFFICE",
"SPRINGFIELD BRANCH OFFICE",
"VERMONT DISTRICT OFFICE",
"CONNECTICUT DISTRICT OFFICE",
"RHODE ISLAND DISTRICT OFFICE",
"MAINE DISTRICT OFFICE",
"NEW HAMPSHIRE DISTRICT OFFICE",
"NEW YORK REGIONAL OFFICE",
"NEW YORK DISTRICT OFFICE",
"ELMIRA BRANCH OFFICE",
"ROCHESTER BRANCH OFFICE",
"MELVILLE BRANCH OFFICE",
"SYRACUSE DISTRICT OFFICE",
"PUERTO RICO AND USVI DISTRICT OFFICE",
"BUFFALO DISTRICT OFFICE",
"NEW JERSEY DISTRICT OFFICE",
"PHILADELPHIA REGIONAL OFFICE",
"PHILADELPHIA DISTRICT OFFICE",
"RICHMOND DISTRICT OFFICE",
"HARRISBURG BRANCH OFFICE",
"WILKES BARRE BRANCH OFFICE",
"CHARLESTON BRANCH OFFICE",
"DELAWARE DISTRICT OFFICE",
"WASHINGTON DISTRICT OFFICE",
"PITTSBURGH DISTRICT OFFICE",
"BALTIMORE DISTRICT OFFICE",
"WEST VIRGINIA DISTRICT OFFICE",
"ATLANTA REGIONAL OFFICE",
"GEORGIA DISTRICT OFFICE",
"GULFPORT",
"SOUTH FLORIDA DISTRICT OFFICE",
"KENTUCKY DISTRICT OFFICE",
"ALABAMA DISTRICT OFFICE",
"NORTH CAROLINA DISTRICT OFFICE",
"SOUTH CAROLINA DISTRICT OFFICE",
"MISSISSIPPI DISTRICT OFFICE",
"TENNESSEE DISTRICT OFFICE",
"NORTH FLORIDA DISTRICT OFFICE",
"CHICAGO REGIONAL OFFICE",
"ILLINOIS DISTRICT OFFICE",
"MINNESOTA DISTRICT OFFICE",
"MICHIGAN DISTRICT OFFICE",
"SPRINGFIELD BRANCH OFFICE",
"CINCINNATI BRANCH OFFICE",
"MARQUETTE BRANCH OFFICE",
"CLEVELAND DISTRICT OFFICE",
"INDIANA DISTRICT OFFICE",
"WISCONSIN DISTRICT OFFICE",
"COLUMBUS DISTRICT OFFICE",
"DALLAS REGIONAL OFFICE",
"DALLAS / FT WORTH DISTRICT OFFICE",
"CORPUS CHRISTI BRANCH OFFICE",
"LOWER RIO GRANDE VALLEY DISTRICT OFFICE",
"ARKANSAS DISTRICT OFFICE",
"HOUSTON DISTRICT OFFICE",
"EL PASO DISTRICT OFFICE",
"LUBBOCK DISTRICT OFFICE",
"LOUISIANA DISTRICT OFFICE",
"OKLAHOMA DISTRICT OFFICE",
"SAN ANTONIO DISTRICT OFFICE",
"NEW MEXICO DISTRICT OFFICE",
"KANSAS CITY REGIONAL OFFICE",
"KANSAS CITY DISTRICT OFFICE",
"SPRINGFIELD BRANCH OFFICE",
"CEDAR RAPIDS BRANCH OFFICE",
"DES MOINES DISTRICT OFFICE",
"NEBRASKA DISTRICT OFFICE",
"WICHITA DISTRICT OFFICE",
"ST. LOUIS DISTRICT OFFICE",
"DENVER REGIONAL OFFICE",
"COLORADO DISTRICT OFFICE",
"NORTH DAKOTA DISTRICT OFFICE",
"SOUTH DAKOTA DISTRICT OFFICE",
"UTAH DISTRICT OFFICE",
"MONTANA DISTRICT OFFICE",
"WYOMING DISTRICT OFFICE",
"SAN FRANCISCO REGIONAL OFFICE",
"SAN FRANCISCO DISTRICT OFFICE",
"LOS ANGELES DISTRICT OFFICE",
"SANTA ANA DISTRICT OFFICE",
"SACRAMENTO DISTRICT OFFICE",
"FRESNO DISTRICT OFFICE",
"NEVADA DISTRICT OFFICE",
"HAWAII DISTRICT OFFICE",
"SAN DIEGO DISTRICT OFFICE",
"ARIZONA DISTRICT OFFICE",
"GUAM BRANCH OFFICE",
"SEATTLE REGIONAL OFFICE",
"SEATTLE DISTRICT OFFICE",
"ALASKA DISTRICT OFFICE",
"PORTLAND DISTRICT OFFICE",
"BOISE DISTRICT OFFICE",
"SPOKANE BRANCH OFFICE"
]
ppp_df_23['SBAOfficeLabel'] = np.select(conditions, values)

ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,Estimated_LenderProfit,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry,SBAOfficeLabel
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,23080.76,20200501,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing,SOUTH CAROLINA DISTRICT OFFICE
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,22107.83,20200501,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...,SOUTH CAROLINA DISTRICT OFFICE
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,20740.65,20200501,20210210,Not Available,NaN,Not Available,0000,00,Not Available,SEATTLE DISTRICT OFFICE
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,14996.13,20200501,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction,SANTA ANA DISTRICT OFFICE
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,11023.11,20200501,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing,MASSACHUSETTS DISTRICT OFFICE


In [37]:
ppp_df_23['SBAOfficeLabel'].nunique()

74

Determining Approval Ranges for Each Loan

In [38]:
loan_conditions = [
(ppp_df_23.CurrentApprovalAmount <= 350000) & (ppp_df_23.CurrentApprovalAmount >= 150000),
(ppp_df_23.CurrentApprovalAmount <= 1000000) & (ppp_df_23.CurrentApprovalAmount > 350000),
(ppp_df_23.CurrentApprovalAmount <= 2000000) & (ppp_df_23.CurrentApprovalAmount > 1000000),
(ppp_df_23.CurrentApprovalAmount <= 5000000) & (ppp_df_23.CurrentApprovalAmount > 2000000),
(ppp_df_23.CurrentApprovalAmount <= 10000000) & (ppp_df_23.CurrentApprovalAmount > 5000000),  
]
loan_values = ["a $150,000-350,000","b $350,000-1 million", "c $1-2 million","d $2-5 million", "e $5-10 million"]
ppp_df_23['Loan_Range'] = np.select(loan_conditions, loan_values)
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,DateApprovedTS,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,20200501,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million"
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,20200501,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million"
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,20200501,20210210,Not Available,NaN,Not Available,0000,00,Not Available,SEATTLE DISTRICT OFFICE,"b $350,000-1 million"
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,20200501,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million"
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,20200501,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million"


Determining Job Report Ranges for each Loan

In [39]:
ppp_df_23["JobsReported"].dtype

dtype('float64')

In [40]:
ppp_df_23.loc[ppp_df_23["JobsReported"] == "0", 'JobsReported'] = 0

In [41]:
#possible use in NAICS division comparison, divisions based on Census of Employment and Wages - BLS - https://data.bls.gov/cew/apps/data_views/data_views.htm#tab=Tables
job_conditions = [
(ppp_df_23.JobsReported <= 5),
(ppp_df_23.JobsReported <= 9) & (ppp_df_23.JobsReported > 5),
(ppp_df_23.JobsReported <= 19) & (ppp_df_23.JobsReported > 9),
(ppp_df_23.JobsReported <= 49) & (ppp_df_23.JobsReported > 19),
(ppp_df_23.JobsReported <= 99) & (ppp_df_23.JobsReported > 49), 
(ppp_df_23.JobsReported <= 249) & (ppp_df_23.JobsReported > 99),
(ppp_df_23.JobsReported <= 499) & (ppp_df_23.JobsReported > 249),
(ppp_df_23.JobsReported > 499)
]
job_values = ["Fewer than 5","5 to 9", "10 to 19","20 to 49", "50 to 99", "100 to 249", "250 to 499", "500 or more"]
ppp_df_23['job_Range'] = np.select(job_conditions, job_values)
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,ForgivenessDateTS,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,20201120,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,20210812,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,20210210,Not Available,NaN,Not Available,0000,00,Not Available,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,20210910,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,20210408,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49


In [42]:
ppp_df_23.loc[ppp_df_23["job_Range"] == "0", 'job_Range'] = "Fewer than 5"
ppp_df_23['job_Range'].value_counts()

20 to 49        416149
10 to 19        241148
50 to 99        161163
100 to 249       84575
5 to 9           31080
250 to 499       21987
Fewer than 5      5093
500 or more       4357
Name: job_Range, dtype: int64

In [43]:
ppp_df_23['job_Range'].isnull().sum() #check 

0

Combining Columns for Full Address 

In [44]:
ppp_df_23['full_add'] = ppp_df_23.BorrowerAddress + ", " + ppp_df_23.BorrowerCity + ", " + ppp_df_23.BorrowerState
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,Not Available,NaN,Not Available,0000,00,Not Available,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,NaN
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,NaN


Determining Business Age & Type Naming Consistency

In [45]:
ppp_df_23["BusinessType"] = ppp_df_23["BusinessType"].replace({'Limited  Liability Company(LLC)': 'Limited Liability Company(LLC)'})


In [46]:
ppp_df_23['BusinessAgeDescription'] = ppp_df_23['BusinessAgeDescription'].fillna('Unanswered')
ppp_df_23["BusinessType"] = ppp_df_23["BusinessType"].fillna('Not Available')
ppp_df_23["NonProfit"] = ppp_df_23["NonProfit"].fillna('N')
ppp_df_23.head()

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add
0,9547507704,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,...,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255,32,Manufacturing,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
1,9777677704,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,Paid in Full,...,5617 Services to Buildings and Dwellings,999 Other,5617 Services to Buildings and Dwellings,5617,56,Administrative and Support and Waste Managemen...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
2,5791407702,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,...,Not Available,NaN,Not Available,0000,00,Not Available,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,NaN
3,6223567700,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,Paid in Full,...,2361 Residential Building Construction,999 Other,2361 Residential Building Construction,2361,23,Construction,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,NaN
4,9662437702,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,Paid in Full,...,4842 Specialized Freight Trucking,999 Other,4842 Specialized Freight Trucking,4842,48,Transportation and Warehousing,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,NaN


In [47]:
listbusiness =ppp_df_23["BusinessType"].value_counts()
listbusiness

Corporation                            415684
Limited Liability Company(LLC)         261058
Subchapter S Corporation               174799
Non-Profit Organization                 55904
Partnership                             18158
Limited Liability Partnership           12724
Sole Proprietorship                     12255
Professional Association                 6270
Cooperative                              2435
501(c)3 – Non Profit                     1772
Non-Profit Childcare Center               900
501(c)6 – Non Profit Membership           762
Not Available                             720
Employee Stock Ownership Plan(ESOP)       641
Self-Employed Individuals                 494
Trust                                     397
Housing Co-op                             152
Joint Venture                             125
Independent Contractors                   112
Single Member LLC                          78
Tribal Concerns                            64
Tenant in Common                  

Dropping Unnecessary Columns 

In [48]:
#keep track of columns removed
dele = ['Unnamed: 0', 'BorrowerAddress', 'BorrowerCity', 'BorrowerZip', 'SBAGuarantyPercentage', 'InitialApprovalAmount', 
 'BusinessAgeDescription', 'ProjectCity','ProjectCountyName', 'ProjectState', 'ProjectZip','Race', 'Ethnicity', 'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState', 'ServicingLenderZip', 
 'UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'DateApproved_ts', 'ForgivenessDate_ts',
 'HEALTH_CARE_PROCEED','DEBT_INTEREST_PROCEED', 'Gender', 'Veteran']
cols_pipe =['LoanNumber', 'DateApprovedTS', 'SBAOfficeCode', 'SBAOfficeLabel','ProcessingMethod', 'BorrowerName', 
       'BorrowerState', 'LoanStatusDate', 'LoanStatus', 'Term', 'CurrentApprovalAmount', 'UndisbursedAmount', 
       'FranchiseName','ServicingLenderLocationID', 'ServicingLenderName', 'RuralUrbanIndicator', 'HubzoneIndicator',  'LMIIndicator', 
       'CD', 'JobsReported', 'OriginatingLenderLocationID', 'OriginatingLender', 'OriginatingLenderCity', 'OriginatingLenderState',
       'BusinessType', 'BusinessAgeDescription', 'NonProfit', 'ForgivenessAmount', 'ForgivenessDateTS', 'Estimated_LenderPercent', 'Estimated_LenderProfit', 
       'NA', '4DigitNAICS', 'NAICSCode', 'Industry', 'IndustrySubsector', 'TRIIndustrySector', 'Loan_Range', 'job_Range',
       'LongName', 'full_add']

ppp_df_23 = ppp_df_23[cols_pipe]
ppp_df_23.head()

,LoanNumber,DateApprovedTS,SBAOfficeCode,SBAOfficeLabel,ProcessingMethod,BorrowerName,BorrowerState,LoanStatusDate,LoanStatus,Term,...,NA,4DigitNAICS,NAICSCode,Industry,IndustrySubsector,TRIIndustrySector,Loan_Range,job_Range,LongName,full_add
0,9547507704,20200501,464,SOUTH CAROLINA DISTRICT OFFICE,PPP,"SUMTER COATINGS, INC.",NaN,12/18/2020,Paid in Full,24,...,32,3255,325510,Manufacturing,"3255 Paint, Coating, and Adhesive Manufacturing",325 Chemicals,"b $350,000-1 million",50 to 99,"3255 Paint, Coating, and Adhesive Manufacturing",NaN
1,9777677704,20200501,464,SOUTH CAROLINA DISTRICT OFFICE,PPP,"PLEASANT PLACES, INC.",NaN,09/28/2021,Paid in Full,24,...,56,5617,561730,Administrative and Support and Waste Managemen...,5617 Services to Buildings and Dwellings,999 Other,"b $350,000-1 million",50 to 99,5617 Services to Buildings and Dwellings,NaN
2,5791407702,20200501,1013,SEATTLE DISTRICT OFFICE,PPP,BOYER CHILDREN'S CLINIC,NaN,03/17/2021,Paid in Full,24,...,00,0000,0,Not Available,Not Available,NaN,"b $350,000-1 million",50 to 99,Not Available,NaN
3,6223567700,20200501,920,SANTA ANA DISTRICT OFFICE,PPP,KIRTLEY CONSTRUCTION INC,NaN,10/16/2021,Paid in Full,24,...,23,2361,236115,Construction,2361 Residential Building Construction,999 Other,"b $350,000-1 million",20 to 49,2361 Residential Building Construction,NaN
4,9662437702,20200501,101,MASSACHUSETTS DISTRICT OFFICE,PPP,AERO BOX LLC,NaN,08/17/2021,Paid in Full,24,...,48,4842,484210,Transportation and Warehousing,4842 Specialized Freight Trucking,999 Other,"b $350,000-1 million",20 to 49,4842 Specialized Freight Trucking,NaN


Parsing State Files

In [49]:
state_counts = ppp_df_23["BorrowerState"].value_counts()
print(state_counts)

CA    129085
TX     76061
NY     73819
FL     60002
IL     40763
PA     38500
OH     32639
NJ     32515
MI     29791
MA     27179
GA     25787
WA     23619
VA     23224
NC     22796
MN     19720
CO     19669
MD     18955
WI     17346
IN     16594
MO     16339
TN     15864
AZ     15757
LA     13669
OR     13403
CT     12882
AL     10912
SC     10739
OK      9995
UT      9258
KY      9178
IA      8295
NV      8254
KS      8201
AR      5857
NE      5804
MS      5504
NH      5069
HI      4979
NM      4482
ID      4415
DC      4383
ME      4185
RI      3786
WV      3365
ND      3222
MT      3142
DE      2990
PR      2952
SD      2695
AK      2662
VT      2265
WY      2137
GU       443
VI       290
MP        82
AS        20
Name: BorrowerState, dtype: int64


In [50]:
ppp_df_23.to_csv("../data/ppp_pipe23.csv")

State Pipe Files (removed 07/19)

In [ ]:
ak_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "AK", :]
al_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "AL", :]
ar_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "AR", :]
as_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "AS", :] #American Samoa
az_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "AZ", :]
ca_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "CA", :]
co_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "CO", :]
ct_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "CT", :]
dc_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "DC", :] #D.C.
de_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "DE", :]
fl_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "FL", :]
ga_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "GA", :]
gu_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "GU", :] #Guam
hi_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "HI", :]
ia_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "IA", :]
id_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "ID", :]
il_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "IL", :]
in_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "IN", :]
ks_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "KS", :]
ky_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "KY", :]
la_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "LA", :]
ma_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MA", :]
md_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MD", :]
me_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "ME", :]
mi_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MI", :]
mn_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MN", :]
mo_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MO", :]
mp_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MP", :] #Saipan Member of Parliament
ms_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MS", :]
mt_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "MT", :]
nc_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NC", :]
nd_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "ND", :]
ne_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NE", :]
nh_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NH", :]
nj_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NJ", :]
nm_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NM", :]
nv_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NV", :]
ny_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "NY", :]
oh_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "OH", :]
ok_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "OK", :]
or_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "OR", :]
pa_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "PA", :]
pr_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "PR", :] #Puerto Rico
ri_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "RI", :]
sc_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "SC", :]
sd_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "SD", :]
tn_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "TN", :]
tx_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "TX", :]
ut_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "UT", :]
va_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "VA", :]
vi_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "VI", :] #Virgin Islands
vt_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "VT", :]
wa_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "WA", :]
wi_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "WI", :]
wv_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "WV", :]
wy_df = ppp_df_23.loc[ppp_df_23["BorrowerState"] == "WY", :]

In [ ]:
ak_df.to_csv("../data/state_data/pipe_states/23/ak.csv") 
al_df.to_csv("../data/state_data/pipe_states/23/al.csv")
ar_df.to_csv("../data/state_data/pipe_states/23/ar.csv") 
as_df.to_csv("../data/state_data/pipe_states/23/as.csv")
az_df.to_csv("../data/state_data/pipe_states/23/az.csv")  
ca_df.to_csv("../data/state_data/pipe_states/23/ca.csv")  
co_df.to_csv("../data/state_data/pipe_states/23/co.csv") 
ct_df.to_csv("../data/state_data/pipe_states/23/ct.csv")  
dc_df.to_csv("../data/state_data/pipe_states/23/dc.csv")
de_df.to_csv("../data/state_data/pipe_states/23/de.csv") 
fl_df.to_csv("../data/state_data/pipe_states/23/fl.csv") 
ga_df.to_csv("../data/state_data/pipe_states/23/ga.csv") 
gu_df.to_csv("../data/state_data/pipe_states/23/gu.csv")
hi_df.to_csv("../data/state_data/pipe_states/23/hi.csv") 
ia_df.to_csv("../data/state_data/pipe_states/23/ia.csv")  
id_df.to_csv("../data/state_data/pipe_states/23/id.csv")  
il_df.to_csv("../data/state_data/pipe_states/23/il.csv")  
in_df.to_csv("../data/state_data/pipe_states/23/in.csv") 
ks_df.to_csv("../data/state_data/pipe_states/23/ks.csv")  
ky_df.to_csv("../data/state_data/pipe_states/23/ky.csv")  
la_df.to_csv("../data/state_data/pipe_states/23/la.csv")
ma_df.to_csv("../data/state_data/pipe_states/23/ma.csv") 
md_df.to_csv("../data/state_data/pipe_states/23/md.csv")
me_df.to_csv("../data/state_data/pipe_states/23/me.csv")  
mi_df.to_csv("../data/state_data/pipe_states/23/mi.csv") 
mn_df.to_csv("../data/state_data/pipe_states/23/mn.csv") 
mo_df.to_csv("../data/state_data/pipe_states/23/mo.csv")  
mp_df.to_csv("../data/state_data/pipe_states/23/mp.csv") 
ms_df.to_csv("../data/state_data/pipe_states/23/ms.csv")
mt_df.to_csv("../data/state_data/pipe_states/23/mt.csv")
nc_df.to_csv("../data/state_data/pipe_states/23/nc.csv") 
nd_df.to_csv("../data/state_data/pipe_states/23/nd.csv")  
ne_df.to_csv("../data/state_data/pipe_states/23/ne.csv")
nh_df.to_csv("../data/state_data/pipe_states/23/nh.csv")  
nj_df.to_csv("../data/state_data/pipe_states/23/nj.csv")
nm_df.to_csv("../data/state_data/pipe_states/23/nm.csv") 
nv_df.to_csv("../data/state_data/pipe_states/23/nv.csv")  
ny_df.to_csv("../data/state_data/pipe_states/23/ny.csv")  
oh_df.to_csv("../data/state_data/pipe_states/23/oh.csv")
ok_df.to_csv("../data/state_data/pipe_states/23/ok.csv") 
or_df.to_csv("../data/state_data/pipe_states/23/or.csv")  
pa_df.to_csv("../data/state_data/pipe_states/23/pa.csv")
pr_df.to_csv("../data/state_data/pipe_states/23/pr.csv")  
ri_df.to_csv("../data/state_data/pipe_states/23/ri.csv") 
sc_df.to_csv("../data/state_data/pipe_states/23/sc.csv") 
sd_df.to_csv("../data/state_data/pipe_states/23/sd.csv") 
tn_df.to_csv("../data/state_data/pipe_states/23/tn.csv")  
tx_df.to_csv("../data/state_data/pipe_states/23/tx.csv")  
ut_df.to_csv("../data/state_data/pipe_states/23/ut.csv")  
va_df.to_csv("../data/state_data/pipe_states/23/va.csv")  
vi_df.to_csv("../data/state_data/pipe_states/23/vi.csv")  
vt_df.to_csv("../data/state_data/pipe_states/23/vt.csv")
wa_df.to_csv("../data/state_data/pipe_states/23/wa.csv")  
wi_df.to_csv("../data/state_data/pipe_states/23/wi.csv")  
wv_df.to_csv("../data/state_data/pipe_states/23/wv.csv") 
wy_df.to_csv("../data/state_data/pipe_states/23/wy.csv")  

Possible Additions to Pipeline:<br>
Location-based:    http://www.city-data.com/ <br>
Dates:
using deep learning with time series data is to split the date into multiple categorical variables (year, month, week, day of week, day of month, and Booleans for whether it’s the start/end of a month/quarter/year). 